
# 🏠 EDA y Modelo de Precios — Rentas Mérida

Este notebook limpia el dataset, genera un EDA básico y entrena dos modelos de predicción de precio (Regresión Lineal y Random Forest).


In [ ]:

import pandas as pd, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

CSV = Path(r"/mnt/data/rentarenmerida.csv")
df_raw = pd.read_csv(CSV)
df_raw.head()


In [ ]:

def to_int(series, default=np.nan):
    s = series.astype(str).str.extract(r'(-?\d+)')[0]
    return pd.to_numeric(s, errors="coerce").fillna(default).astype("Int64")

def money_to_float(series):
    s = series.astype(str).str.replace(r'[^0-9\.-]', '', regex=True)
    return pd.to_numeric(s, errors='coerce')


In [ ]:

df = df_raw.copy()
df.columns = [c.strip().lower() for c in df.columns]
df["bedroom"]  = to_int(df["bedroom"])
df["bathroom"] = to_int(df["bathroom"])
df["parking"]  = to_int(df["parking"])
df["area"]     = pd.to_numeric(df["area"], errors="coerce").astype("Int64")
df["price"]    = money_to_float(df["price"])

df = df.dropna(subset=["price","area"]).copy()
df.head()


In [ ]:

# IQR outlier removal for price and area
def iqr_clip(s):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    return lo, hi

p_lo, p_hi = iqr_clip(df["price"])
a_lo, a_hi = iqr_clip(df["area"].astype(float))

df = df[(df["price"]>=p_lo) & (df["price"]<=p_hi) &
        (df["area"].astype(float)>=a_lo) & (df["area"].astype(float)<=a_hi)].copy()
df.describe().T


In [ ]:

# Hist price
plt.figure()
df["price"].plot(kind="hist", bins=30)
plt.title("Distribución de precios de renta")
plt.xlabel("Precio"); plt.ylabel("Frecuencia")
plt.tight_layout(); plt.show()

# Scatter area vs price
plt.figure()
plt.scatter(df["area"].astype(float), df["price"])
plt.title("Precio vs Área (m²)")
plt.xlabel("Área (m²)"); plt.ylabel("Precio")
plt.tight_layout(); plt.show()


In [ ]:

# Modeling
data = df.copy()
for c in ["bedroom","bathroom","parking"]:
    data[c] = data[c].fillna(data[c].median())

X = data[["area","bedroom","bathroom","parking"]].astype(float)
y = data["price"].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression().fit(X_train, y_train)
rf = RandomForestRegressor(n_estimators=300, random_state=42).fit(X_train, y_train)

pred_lr = lr.predict(X_test)
pred_rf = rf.predict(X_test)

def metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

m_lr = metrics(y_test, pred_lr)
m_rf = metrics(y_test, pred_rf)

print("LinearRegression:", m_lr)
print("RandomForestRegressor:", m_rf)


In [ ]:

# Plot predicted vs actual (RF)
plt.figure()
plt.scatter(y_test, pred_rf)
plt.title("RandomForest — Precio real vs. predicho")
plt.xlabel("Precio real"); plt.ylabel("Precio predicho")
plt.tight_layout(); plt.show()
